In [6]:
import pandas as pd

# Assuming your dataset is in a CSV file
file_path = r'inference_drd\Diabetic_Retinopathy_Detection\train.csv'

# Read the dataset
df = pd.read_csv(file_path)

# List of columns to drop
columns_to_drop = ['Unnamed: 0', 'path', 'exists']  # Add any other columns you want to drop

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

# Function to sample 30 records from each level
def sample_and_remove(group, n=30):
    sampled = group.sample(n=min(len(group), n), random_state=1)
    remaining = group.drop(sampled.index)
    return sampled, remaining

# Initialize empty lists to store sampled and remaining DataFrames
sampled_list = []
remaining_list = []

# Group by 'level' and apply the sampling function
for name, group in df.groupby('level'):
    sampled, remaining = sample_and_remove(group)
    sampled_list.append(sampled)
    remaining_list.append(remaining)

# Concatenate the sampled and remaining DataFrames
sampled_df = pd.concat(sampled_list, ignore_index=True)
remaining_df = pd.concat(remaining_list, ignore_index=True)

# Save or display the sampled DataFrame
sampled_df.to_csv('sampled_dataset.csv', index=False)
print("Sampled records:")
print(sampled_df)

# Save or display the remaining DataFrame
remaining_df.to_csv('remaining_dataset.csv', index=False)
print("\nRemaining records:")
print(remaining_df)


Sampled records:
           image  level  PatientId  eye         level_cat      HbA1c  \
0      9103_left      0       9103    1  [1. 0. 0. 0. 0.]   5.859674   
1     12948_left      0      12948    1  [1. 0. 0. 0. 0.]   5.608829   
2     15859_left      0      15859    1  [1. 0. 0. 0. 0.]   6.569314   
3    19968_right      0      19968    0  [1. 0. 0. 0. 0.]   6.636184   
4      4197_left      0       4197    1  [1. 0. 0. 0. 0.]   6.334123   
..           ...    ...        ...  ...               ...        ...   
145  33080_right      4      33080    0  [0. 0. 0. 0. 1.]  10.094568   
146  32148_right      4      32148    0  [0. 0. 0. 0. 1.]  10.364584   
147   3563_right      4       3563    0  [0. 0. 0. 0. 1.]  10.547997   
148  13664_right      4      13664    0  [0. 0. 0. 0. 1.]  10.322404   
149  39081_right      4      39081    0  [0. 0. 0. 0. 1.]  10.825708   

     Systolic_BP  Diastolic_BP         LDL   Duration        BMI  Glucose_SD  \
0     116.050591     75.564766   70.64

In [3]:
df = pd.read_csv('./inference_drd/Diabetic_Retinopathy_Detection/train.csv')
df.head()

,Unnamed: 0,image,level,PatientId,path,exists,eye,level_cat,HbA1c,Systolic_BP,...,LDL,Duration,BMI,Glucose_SD,Triglycerides,Microalbuminuria,Smoking_years,Alcohol_frequency,BP,full_path
0,0,11204_right,0,11204,.\train.zip.001\train\train\11204_right.jpeg,True,0,[1. 0. 0. 0. 0.],6.664607,118.160623,...,95.632704,3.173715,23.683483,13.023127,116.933755,8.170038,4,0,113.702971,E:\Computer_Vision_projects\Diabetic_retinopat...
1,1,12793_right,0,12793,.\train.zip.001\train\train\12793_right.jpeg,True,0,[1. 0. 0. 0. 0.],5.569154,114.915966,...,98.923421,1.792701,23.139272,12.654202,56.645795,2.261660,0,0,112.601676,E:\Computer_Vision_projects\Diabetic_retinopat...
2,2,10614_right,0,10614,.\train.zip.001\train\train\10614_right.jpeg,True,0,[1. 0. 0. 0. 0.],5.673113,114.006031,...,81.936662,4.964299,23.275760,17.254322,60.284675,16.585806,0,0,119.270236,E:\Computer_Vision_projects\Diabetic_retinopat...
3,3,10046_right,0,10046,.\train.zip.001\train\train\10046_right.jpeg,True,0,[1. 0. 0. 0. 0.],5.819946,119.703647,...,83.784645,2.611829,21.164522,12.427103,112.322626,7.779978,4,0,115.776956,E:\Computer_Vision_projects\Diabetic_retinopat...
4,4,17936_right,0,17936,.\train.zip.001\train\train\17936_right.jpeg,True,0,[1. 0. 0. 0. 0.],6.943961,113.874062,...,97.558886,3.736780,21.516293,16.995806,55.465491,20.792241,0,0,117.560609,E:\Computer_Vision_projects\Diabetic_retinopat...


In [4]:
df.columns

Index(['Unnamed: 0', 'image', 'level', 'PatientId', 'path', 'exists', 'eye',
       'level_cat', 'HbA1c', 'Systolic_BP', 'Diastolic_BP', 'LDL', 'Duration',
       'BMI', 'Glucose_SD', 'Triglycerides', 'Microalbuminuria',
       'Smoking_years', 'Alcohol_frequency', 'BP', 'full_path'],
      dtype='object')

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle 

# Load the CSV file
csv_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\remaining_dataset.csv'
df = pd.read_csv(csv_file)

# Fix the format of level_cat and extract labels
def fix_level_cat_format(row):
    return eval(row.replace(' ', ', '))

labels = np.array([fix_level_cat_format(row) for row in df['level_cat'].values])

# Extract additional features
additional_features = df[['HbA1c', 'Systolic_BP', 'Diastolic_BP', 'LDL', 'Duration', 
                          'BMI', 'Glucose_SD', 'Triglycerides', 'Microalbuminuria', 
                          'Smoking_years', 'Alcohol_frequency']].values

# Normalize additional features
scaler = StandardScaler()
additional_features = scaler.fit_transform(additional_features)

# Save the scaler object
scaler_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\additional_features_scaler.pkl'
with open(scaler_file, 'wb') as f:
    pickle.dump(scaler, f)

# Load and preprocess images
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

images = np.array([load_and_preprocess_image(path) for path in df['full_path']])

# Split the data into training and testing sets
X_train_img, X_test_img, X_train_feat, X_test_feat, y_train, y_test = train_test_split(
    images, additional_features, labels, test_size=0.2, random_state=42
)

### Step 2: Modify the VGG16 Model

# Load VGG16 model without the top layers
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = vgg16.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
vgg_output = Dense(128, activation='relu')(x)

# Define additional features input
additional_input = Input(shape=(additional_features.shape[1],))

### Step 3: Combine the VGG16 Output and Additional Features

# Combine VGG16 output and additional features
combined = concatenate([vgg_output, additional_input])

# Add more dense layers if needed
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.5)(combined)
final_output = Dense(5, activation='softmax')(combined)  # Assuming five classes for multi-class classification

# Create the final model
model = Model(inputs=[vgg16.input, additional_input], outputs=final_output)

### Step 4: Compile and Train the Model

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    [X_train_img, X_train_feat], y_train,
    validation_data=([X_test_img, X_test_feat], y_test),
    epochs=12,  # Increased number of epochs to make use of EarlyStopping
    batch_size=64,
    callbacks=[reduce_lr, early_stopping]
)

### Step 5: Evaluate and Save the Model

# Evaluate the model
loss, accuracy = model.evaluate([X_test_img, X_test_feat], y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


model.save(r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\vgg16_with_additional_features.h5')


Epoch 1/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2072s 61s/step - accuracy: 0.2270 - loss: 2.9909 - val_accuracy: 0.3060 - val_loss: 1.5227 - learning_rate: 1.0000e-04
Epoch 2/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2050s 60s/step - accuracy: 0.2898 - loss: 1.5439 - val_accuracy: 0.3899 - val_loss: 1.3522 - learning_rate: 1.0000e-04
Epoch 3/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2191s 65s/step - accuracy: 0.3934 - loss: 1.3843 - val_accuracy: 0.5728 - val_loss: 1.2044 - learning_rate: 1.0000e-04
Epoch 4/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2415s 71s/step - accuracy: 0.4342 - loss: 1.2950 - val_accuracy: 0.7239 - val_loss: 1.1149 - learning_rate: 1.0000e-04
Epoch 5/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2630s 78s/step - accuracy: 0.5018 - loss: 1.2098 - val_accuracy: 0.6959 - val_loss: 1.0707 - learning_rate: 1.0000e-04
Epoch 6/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2774s 82s/step - accuracy: 0.5542 - loss: 1.1388 - val_accuracy: 0.6362 - val_loss: 1.0226 - learning_rate: 1.0000e-04
Epoch 7/12
34/34 ━━━━━━━━━━━━━━━━━━━━ 2456s 72s/step - acc

Loss: 0.7512641549110413, Accuracy: 0.9384328126907349


In [1]:
import os
import warnings
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler

# Suppress TensorFlow logs and oneDNN custom operations
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress all TensorFlow logs
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN custom operations

# Suppress all warnings
warnings.filterwarnings('ignore')

# Further suppress TensorFlow logging
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

# Load your saved model
model = tf.keras.models.load_model(r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\vgg16_with_additional_features.h5', custom_objects=None, compile=True, safe_mode=True)

# Function to load and preprocess image
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Function to load the scaler
def load_scaler(scaler_file):
    with open(scaler_file, 'rb') as f:
        scaler = pickle.load(f)
    return scaler

# Function to predict class for a single record
def predict_single_record(left_eye_path, right_eye_path, additional_features, scaler):
    # Load and preprocess images
    left_eye_img = load_and_preprocess_image(left_eye_path)
    right_eye_img = load_and_preprocess_image(right_eye_path)

    # Convert additional features to numpy array and scale using loaded scaler
    additional_features = np.array(additional_features).reshape(1, -1)
    additional_features_scaled = scaler.transform(additional_features)

    # Repeat additional features to match the number of images (left and right eye)
    additional_features_repeated = np.repeat(additional_features_scaled, 2, axis=0)

    # Stack images to create a batch of size 2 (left and right eye)
    images = np.stack([left_eye_img, right_eye_img], axis=0)

    # Temporarily suppress warnings during prediction
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Predict for both eyes
        predictions = model.predict([images, additional_features_repeated])
    
    # Assuming predictions are probabilities for each class, extract the predicted class and confidence
    predicted_classes = np.argmax(predictions, axis=1)
    confidences = np.max(predictions, axis=1)
    
    return predicted_classes, confidences

# Paths to left and right eye images
left_eye_path = 'E:/Computer_Vision_projects/Diabetic_retinopathy_detection/train.zip.001/train/train/10_left.jpeg'
right_eye_path = 'E:/Computer_Vision_projects/Diabetic_retinopathy_detection/train.zip.001/train/train/10_right.jpeg'

# Path to the scaler file
scaler_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\additional_features_scaler.pkl'

# Load the scaler
scaler = load_scaler(scaler_file)

# Example additional features (replace with your own)
additional_features = [
    10.896195065561075,  # HbA1c
    150.69693481299157,  # Systolic_BP
    97.58934460717728,   # Diastolic_BP
    193.71066317092425,  # LDL
    21.41599013207625,   # Duration
    41.20415312086392,   # BMI
    30.84676095056483,   # Glucose_SD
    311.23456504422853,  # Triglycerides
    120.01877115621907,  # Microalbuminuria
    23,                  # Smoking_years
    6,                   # Alcohol_frequency
]

# Get predictions and confidences
predicted_classes, confidences = predict_single_record(left_eye_path, right_eye_path, additional_features, scaler)

# Output predictions and confidences for left and right eye
print(f'Predicted class for left eye: {predicted_classes[0]} with confidence {confidences[0]:.4f}')
print(f'Predicted class for right eye: {predicted_classes[1]} with confidence {confidences[1]:.4f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 888ms/step
Predicted class for left eye: 4 with confidence 0.6738
Predicted class for right eye: 4 with confidence 0.6609


In [2]:
import os
import warnings
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,  confusion_matrix, classification_report

# Suppress TensorFlow logs and oneDNN custom operations
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress all TensorFlow logs
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN custom operations

# Suppress all warnings
warnings.filterwarnings('ignore')

# Further suppress TensorFlow logging
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

# Load your saved model
model = tf.keras.models.load_model(r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\vgg16_with_additional_features.h5')

# Function to load and preprocess image
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Function to load the scaler
def load_scaler(scaler_file):
    with open(scaler_file, 'rb') as f:
        scaler = pickle.load(f)
    return scaler

# Fix the format of level_cat and extract labels
def fix_level_cat_format(row):
    return eval(row.replace(' ', ', '))

# Function to predict class for multiple records
def predict_records(df, scaler):
    images = []
    additional_features = []
    true_labels = []

    for index, row in df.iterrows():
        left_eye_img = load_and_preprocess_image(row['full_path'])
        images.append(left_eye_img)
        additional_features.append([
            row['HbA1c'], row['Systolic_BP'], row['Diastolic_BP'], row['LDL'], 
            row['Duration'], row['BMI'], row['Glucose_SD'], row['Triglycerides'], 
            row['Microalbuminuria'], row['Smoking_years'], row['Alcohol_frequency']
        ])
        true_labels.append(fix_level_cat_format(row['level_cat']))  # Fix the format of level_cat

    # Convert to numpy arrays
    images = np.array(images)
    additional_features = np.array(additional_features)
    true_labels = np.array(true_labels)

    # Scale additional features using the loaded scaler
    additional_features_scaled = scaler.transform(additional_features)

    # Predict using the model
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        predictions = model.predict([images, additional_features_scaled])

    # Extract the predicted classes
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(true_labels, axis=1)

    return true_classes, predicted_classes

# Load the CSV file
csv_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\sampled_dataset.csv'
df = pd.read_csv(csv_file)

# Path to the scaler file
scaler_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\additional_features_scaler.pkl'

# Load the scaler
scaler = load_scaler(scaler_file)

# Predict for all records in the CSV
true_classes, predicted_classes = predict_records(df, scaler)

# Calculate metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification report
class_report = classification_report(true_classes, predicted_classes, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4'])
print('Classification Report:')
print(class_report)

5/5 ━━━━━━━━━━━━━━━━━━━━ 32s 6s/step
Accuracy: 0.9667
Precision: 0.9693
Recall: 0.9667
F1 Score: 0.9666


In [3]:
# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Classification report
class_report = classification_report(true_classes, predicted_classes, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4'])
print('Classification Report:')
print(class_report)

Confusion Matrix:
[[30  0  0  0  0]
 [ 2 28  0  0  0]
 [ 0  0 30  0  0]
 [ 0  0  0 27  3]
 [ 0  0  0  0 30]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.94      1.00      0.97        30
     Class 1       1.00      0.93      0.97        30
     Class 2       1.00      1.00      1.00        30
     Class 3       1.00      0.90      0.95        30
     Class 4       0.91      1.00      0.95        30

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle


def load_csv(csv_file):
    return pd.read_csv(csv_file)

def fix_level_cat_format(row):
    return eval(row.replace(' ', ', '))

def extract_labels(df):
    return np.array([fix_level_cat_format(row) for row in df['level_cat'].values])

def extract_additional_features(df):
    return df[['HbA1c', 'Systolic_BP', 'Diastolic_BP', 'LDL', 'Duration', 
               'BMI', 'Glucose_SD', 'Triglycerides', 'Microalbuminuria', 
               'Smoking_years', 'Alcohol_frequency']].values

def normalize_features(features):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    return scaled_features, scaler

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def load_images(image_paths):
    return np.array([load_and_preprocess_image(path) for path in image_paths])

def split_data(images, additional_features, labels, test_size=0.2, random_state=42):
    return train_test_split(images, additional_features, labels, test_size=test_size, random_state=random_state)

def save_scaler(scaler, scaler_file):
    with open(scaler_file, 'wb') as f:
        pickle.dump(scaler, f)

def create_vgg16_model(input_shape=(224, 224, 3)):
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = vgg16.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    vgg_output = Dense(128, activation='relu')(x)
    return vgg16.input, vgg_output

def create_additional_input(input_shape):
    return Input(shape=input_shape)

def create_combined_model(vgg_input, vgg_output, additional_input, num_classes=5):
    combined = concatenate([vgg_output, additional_input])
    combined = Dense(128, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    final_output = Dense(num_classes, activation='softmax')(combined)
    return Model(inputs=[vgg_input, additional_input], outputs=final_output)

def compile_and_train_model(model, X_train_img, X_train_feat, y_train, X_test_img, X_test_feat, y_test, epochs=12, batch_size=64):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(
        [X_train_img, X_train_feat], y_train,
        validation_data=([X_test_img, X_test_feat], y_test),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[reduce_lr, early_stopping]
    )
    return history

def evaluate_and_save_model(model, X_test_img, X_test_feat, y_test, model_save_path):
    loss, accuracy = model.evaluate([X_test_img, X_test_feat], y_test)
    print(f'Loss: {loss}, Accuracy: {accuracy}')
    model.save(model_save_path)


csv_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\remaining_dataset.csv'

scaler_file = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\additional_features_scaler.pkl'

model_save_path = r'E:\Computer_Vision_projects\inference_drd\Diabetic_Retinopathy_Detection\vgg16_with_additional_features.h5'

df = load_csv(csv_file)

labels = extract_labels(df)

additional_features = extract_additional_features(df)

additional_features, scaler = normalize_features(additional_features)

save_scaler(scaler, scaler_file)

images = load_images(df['full_path'])

X_train_img, X_test_img, X_train_feat, X_test_feat, y_train, y_test = split_data(images, additional_features, labels)

vgg_input, vgg_output = create_vgg16_model()

additional_input = create_additional_input((additional_features.shape[1],))

model = create_combined_model(vgg_input, vgg_output, additional_input)

history = compile_and_train_model(model, X_train_img, X_train_feat, y_train, X_test_img, X_test_feat, y_test)

evaluate_and_save_model(model, X_test_img, X_test_feat, y_test, model_save_path)
